In [0]:
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.getcwd()

'/content'

In [0]:
os.listdir()

['.config', 'gdrive', 'sample_data']

In [0]:
root_path = './gdrive/My Drive/OST_Project/working/'

In [0]:
image_height = 217
image_width = 223

In [0]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(image_height,image_width,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 217, 223, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 217, 223, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 215, 221, 64)      18496     
_________________________________________________________________
activation_2 (Activation)    (None, 215, 221, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 107, 110, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)  

In [0]:
train_datagen = ImageDataGenerator(
      rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_dir = root_path + 'train'
validation_dir = root_path + 'validation'

In [0]:
# Change the batchsize according to your system RAM
train_batchsize = 32
val_batchsize = 32
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_height, image_width),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_height, image_width),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 7632 images belonging to 10 classes.
Found 792 images belonging to 10 classes.


In [0]:
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [0]:
# checkpoint
filepath= root_path + "weights_best4.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      callbacks=callbacks_list,
      verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
239/238 [==============================] - 3789s 16s/step - loss: 1.6406 - acc: 0.4253 - val_loss: 1.5693 - val_acc: 0.4697

Epoch 00001: val_acc improved from -inf to 0.46970, saving model to ./gdrive/My Drive/OST_Project/working/weights_best4.hdf5
Epoch 2/200
239/238 [==============================] - 95s 396ms/step - loss: 0.8935 - acc: 0.7127 - val_loss: 1.3547 - val_acc: 0.5682

Epoch 00002: val_acc improved from 0.46970 to 0.56818, saving model to ./gdrive/My Drive/OST_Project/working/weights_best4.hdf5
Epoch 3/200
239/238 [==============================] - 95s 396ms/step - loss: 0.6046 - acc: 0.8016 - val_loss: 1.5957 - val_acc: 0.5985

Epoch 00003: val_acc improved from 0.56818 to 0.59848, saving model to ./gdrive/My Drive/OST_Project/working/weights_best4.hdf5
Epoch 4/200
239/238 [==============================] - 94s 395ms/step - loss: 0.4370 - acc: 0.8628 - val_loss: 1.9729 - val_acc: 0.5884

Epoch 00004: val_acc di

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()